## Spring Boot Interceptor

Dalam Spring, ketika request dikirim ke controller, request tersebut harus melewati interceptor sebelum diproses oleh controller. Ilustrasinya adalah seperti berikut:

![alt text](https://o7planning.org/en/11689/cache/images/i/14286819.png "Logo Title Text 1")

Ada 3 jenis method untuk interceptor:
1. **preHandle()**: digunakan untuk melakukan operasi sebelum mengirim permintaan (request) ke controller. 
2. **postHandle()**: digunakan untuk melakukan operasi sebelum mengirim respon ke client.
3. **afterCompletion()**: digunakan untuk melakukan operasi setelah menyelesaikan request dan respon

Interceptor harus mengimplement 3 abstract method dari class **org.springframework.web.servlet.handler.HandlerInterceptorAdapter** seperti berikut:

```java
public boolean preHandle(HttpServletRequest request, 
                         HttpServletResponse response, 
                         Object handler)

public void postHandle(HttpServletRequest request, 
                       HttpServletResponse response, 
                       Object handler, 
                       ModelAndView modelAndView)

public void afterCompletion(HttpServletRequest request, 
                            HttpServletResponse response, 
                            Object handler, 
                            Exception ex)
```

Kegunaannya di antaranya:
1. Log
2. Audit trail
3. Language

Setiap request bisa melewati satu interceptor bahkan lebih tergantung kebutuhan. Ilustrasinya adalah sebagai berikut:
1. Request melewati satu interceptor
<img src="https://o7planning.org/en/11689/cache/images/i/14286863.png" alt="Satu Interceptor"
	title="Satu Interceptor" width="400" height="400" />
2. Request melewati lebih dari satu interceptor
<img src="https://o7planning.org/en/11689/cache/images/i/14286832.png" alt="Dua Interceptor"
	title="Dua Interceptor" width="400" height="400" />

## Lombok

Lombok adalah sebuah library open source (standalone jar) yang mampu meng-*generate code boilerplate* untuk java class apapun seperti getter, setter, constructor (all argument dan no argument), hashcode, dsb hanya dengan menambahkan anotasi tertentu pada java class yang dimaksud.

### Instalasi
Cara install lombok pada sts/eclipse adalah sebagai berikut:
1. Download lombok jar file

    Untuk mengunduh melalui maven, pada pom.xml tambahkan code berikut:
    ```
    <dependency>
        <groupId>org.projectlombok</groupId>
        <artifactId>lombok</artifactId>
    </dependency>
    ```
2. Instalasi Lombok
    - Buka jar file lombok, 
    - Masukkan path sts.exe atau eclipse.exe
    - Klik install/update
    
### Penggunaan
Contoh penggunaan lombok pada java class adalah sebagai berikut:
```java
import lombok.AllArgsConstructor;
import lombok.Data;

@Data
@AllArgsConstructor
public class BankAccount {
	private Long id;
	private String firstName;
	private String lastName;
	private Double balance;
	
	public BankAccount() {
		
	}	
}
```

Dengan menambahkan anotasi ```@Data``` dan ```@AllArgsConstructor```, method seperti getter dan setter untuk semua field, ```toString()```, ```hashCode()``` dan constructor akan di-generate secara otomatis dan muncul pada halaman outline seperti berikut:

<img style="float: left;" src="https://imgur.com/mOtcSms.png" alt="Lombok"
	title="Penggunaan Lombok" width="250" height="300" />

## JDBC vs JPA

### Definisi

1. JDBC is a standard for Database Access

    JDBC adalah standar untuk mengakses database secara langsung dan mengeksekusi syntax sql seperti ```SELECT * FROM user```, dsb. Syntax sql yang lain seperti ```INSERT```, ```DELETE```, ```UPDATE``` bahkan Stored Procedure sering digunakan pada JDBC. Kekurangannya adalah syntax sql akan semakin rumit apabila banyak relasi yang digunakan pada aplikasi. 


2. JPA is a standard for ORM (Object Relational Mapping)

    JPA mempunyai kemampuan untuk memetakan antara object pada code dan tabel database. Sehingga apa yang biasa dilakukan dengan syntax sql pada JDBC dapat dilakukan dengan menggunakan syntax java, sehingga programmer dapat fokus pada bahasa Java saja. Selain itu, untuk relasi antar object dapat dilakukan dengan mudah dengan JPA ini, bahkan untuk create table pun dapat dilakukan dengan syntax java. Namun, untuk stored procedure, trigger maupun fungsi-fungsi bawaan dari database akan sulit jika menggunakan ORM.
    
### Perbedaan

Perbedaan | JDBC | JPA
---- | ---- | ----
Tabel Database | Harus dibuat terlebih dahulu pada database | Dapat di-generate menggunakan syntax java 
Syntax CRUD SQL | Masih diperlukan menggunakan syntax SQL | Dapat dibantu dengan ORM menggunakan ```CrudRepository``` 
Multiple database | Sulit diimplementasikan | Mudah diimplementasikan karena menggunakan ORM
Performa | Lebih cepat eksekusinya | Kalah jika dibandingkan JDBC (tapi selisih hanya miliseconds sehingga tidak begitu berarti)

### Implementasi
#### Model
1. JDBC

    Model pada aplikasi yang menerapkan JDBC sama seperti model pada umumnya, seperti berikut:

    ```java
    import lombok.AllArgsConstructor;
    import lombok.Data;

    @Data
    @AllArgsConstructor
    public class BankAccount {
        private Long id;
        private String firstName;
        private String lastName;
        private Double balance;

        public BankAccount() {

        }	
    }
    ```

2. JPA

    Model pada aplikasi yang menerapkan JPA, agak sedikit berbeda karena kemampuannya dalam melakukan generate tabel, sehingga perlu ditambahkan beberapa anotasi seperti ```@Entity```, ```@Table```, ```@Id```,```@GeneratedValue``` dan ```@Column``` \*) seperti berikut:

    \*) *NB: untuk keterangan lengkap terkait anotasi JPA dapat mengakses link berikut https://dzone.com/articles/all-jpa-annotations-mapping-annotations*

    ```java
    import javax.persistence.Column;
    import javax.persistence.Entity;
    import javax.persistence.GeneratedValue;
    import javax.persistence.GenerationType;
    import javax.persistence.Id;
    import javax.persistence.Table;

    import lombok.AllArgsConstructor;
    import lombok.Data;

    @Data
    @AllArgsConstructor

    @Entity
    @Table(name="bank_account")
    public class BankAccount {
        @Id
        @GeneratedValue(strategy = GenerationType.IDENTITY)
        @Column(name="bank_id", nullable = false)
        private Long id;

        @Column(name="first_name", nullable = false, length=100)
        private String firstName;

        @Column(name="last_name", nullable = false, length=100)
        private String lastName;

        @Column(name="balance", nullable = false)
        private Double balance;

        public BankAccount() {

        }	
    }
    ```

#### Service
1. JDBC

    Menggunakan abstract method CRUD yang harus diimplementasi oleh subclass, seperti berikut:
    ```java
    // interface
    import java.util.List;

    import id.go.bps.jdbc.exception.TransferFundException;
    import id.go.bps.jdbc.model.BankAccount;

    public interface BankAccountService {
        public String addBankAccount(BankAccount bankAccount);
        public String editBankAccount(BankAccount bankAccount, Long id);
        public String deleteBankAccount(Long id);
        public BankAccount findBankAccountById(Long id);
        public List<BankAccount> findAll();
        public int addAmount(Long id, Double amount) throws TransferFundException;
        public void TransferFund(Long fromAccount, Long toAccount, Double amount) throws TransferFundException;
    }
    ```
    -------------
    ```java
    // subclass
    @Repository
    public class BankAccountServiceImpl implements BankAccountService{
        @Autowired
        private JdbcTemplate jdbcTemplate;

        public JdbcTemplate getJdbcTemplate() {
            return jdbcTemplate;
        }

        @Override
        public String addBankAccount(BankAccount bankAccount) {
            // TODO Auto-generated method stub
            String sql = "INSERT INTO bank_account(first_name,last_name,balance) "
                    + "VALUE (?,?,?)";
            int result = getJdbcTemplate().update(sql, 
                    bankAccount.getFirstName(), 
                    bankAccount.getLastName(), 
                    bankAccount.getBalance());
            if(result==1) {
                return "Add Record Success";
            }else {
                return "Add Record Failed";
            }
        }

        @Override
        public String editBankAccount(BankAccount bankAccount, Long id) {
            // TODO Auto-generated method stub
            String sql = "UPDATE bank_account "
                    + "SET first_name=?, "
                    + "last_name=?, "
                    + "balance=? WHERE id=?";
            int result = getJdbcTemplate().update(sql, 
                    bankAccount.getFirstName(), 
                    bankAccount.getLastName(), 
                    bankAccount.getBalance(), id);
            if(result==1) {
                return "Update Record Success";
            }else {
                return "Update Record Failed";
            }
        }

        @Override
        public String deleteBankAccount(Long id) {
            // TODO Auto-generated method stub
            String sql = "DELETE bank_account WHERE id=?";
            int result = getJdbcTemplate().update(sql, id);
            if(result==1) {
                return "Delete Record Success";
            }else {
                return "Delete Record Failed";
            }
        }

        @Override
        public BankAccount findBankAccountById(Long id) {
            String sql = "SELECT * FROM bank_account WHERE id=?";
            try {
                BankAccount bankAccount = getJdbcTemplate().queryForObject(sql, 
                        new Object[] {id}, new BeanPropertyRowMapper<>(BankAccount.class));
                return bankAccount;
            } catch (Exception e) {
                return null;
            }
        }

        @Override
        public List<BankAccount> findAll() {
            String sql = "SELECT * FROM bank_account";
            List<BankAccount> bankAccountList = getJdbcTemplate().query(sql, 
                    new BeanPropertyRowMapper<>(BankAccount.class));
            return bankAccountList;
        }

        @Override
        @Transactional(propagation = Propagation.MANDATORY)
        public int addAmount(Long id, Double amount) throws TransferFundException {
            // TODO Auto-generated method stub
            // 1. find BankAccount exist or not
            BankAccount bankAccount = this.findBankAccountById(id);
            if (bankAccount==null) {
                throw new TransferFundException("Bank Account "+id+" not found!");
            }
            // 2. Check balance sufficient or not
            Double newbalance = bankAccount.getBalance() + amount;
            if (bankAccount.getBalance() + amount < 0) {
                throw new TransferFundException("Money in account "+id+" is not enough");
            }
            bankAccount.setBalance(newbalance);
            String sql = "UPDATE bank_account "
                    + "SET balance=? WHERE id=?";
            int result = getJdbcTemplate().update(sql,  
                    bankAccount.getBalance(), id);

            return result;
        }

        @Override
        @Transactional(propagation = Propagation.REQUIRES_NEW, rollbackFor = TransferFundException.class)
        public void TransferFund(Long fromAccount, Long toAccount, Double amount) throws TransferFundException {
            // TODO Auto-generated method stub
            addAmount(fromAccount, -amount);
            addAmount(toAccount, amount);
        }
    }

    ```
2. JPA
    Tinggal ```extends``` ```CrudRepository``` untuk menghasilkan ORM CRUD seperti ```addBankAccount(), editBankAccount(), deleteBankAccount(), findBankAccountById(), dan findAll()``` secara otomatis pada subclass, seperti contoh berikut:
    ```java
    // interface
    import org.springframework.data.repository.CrudRepository;

    import id.go.bps.jpa.model.BankAccount;

    public interface BankAccountService extends CrudRepository<BankAccount, Long>{

    }
    ```
    -------------
    ```java
    // sublass
    @Repository
    public class BankAccountServiceImpl implements TransferFundService{
        @Override
        @Transactional(propagation = Propagation.MANDATORY)
        public int addAmount(Long id, Double amount) throws TransferFundException {
            // TODO Auto-generated method stub
            // 1. find BankAccount exist or not
            BankAccount bankAccount = this.findBankAccountById(id);
            if (bankAccount==null) {
                throw new TransferFundException("Bank Account "+id+" not found!");
            }
            // 2. Check balance sufficient or not
            Double newbalance = bankAccount.getBalance() + amount;
            if (bankAccount.getBalance() + amount < 0) {
                throw new TransferFundException("Money in account "+id+" is not enough");
            }
            bankAccount.setBalance(newbalance);

            return 0;
        }

        @Override
        @Transactional(propagation = Propagation.REQUIRES_NEW, rollbackFor = TransferFundException.class)
        public void TransferFund(Long fromAccount, Long toAccount, Double amount) throws TransferFundException {
            // TODO Auto-generated method stub
            addAmount(fromAccount, -amount);
            addAmount(toAccount, amount);
        }
    }
    ```